# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [6]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [46]:

dates = pd.read_csv('dates/dates_updated.csv')
dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
dates['document_type'] = 'statement' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Statement Date'].min() - pd.Timedelta(days=15)
end_date = dates['Statement Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
statements_df = statements_df[column_order]

statements_df

statements_df.to_csv('raw_data/statement_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_9312/484069958.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*********************

### Problem
Need to figure out how to get the days for around the intermeeting dates now because 
the dates in the html links are the same as the fed statements for those, which would cause a problem 
when getting return data.

How to get around this?
Find a dataset with the dates of the intermeeting or make our own (just manually do it - would take an hour maybe)

Ended up just manually doing this

In [47]:

dates = pd.read_csv('dates/dates_updated.csv')
dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
dates['document_type'] = 'intermeeting' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Intermeeting Date'].min() - pd.Timedelta(days=15)
end_date = dates['Intermeeting Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

intermeeting_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
intermeeting_df = intermeeting_df[column_order]

intermeeting_df

intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_9312/3896584411.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**************

### Need to merge the two datasets

In [52]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df = combined_df.dropna(subset=['announcement_date'])
combined_df = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_df['meeting_id'] = combined_df['announcement_date'].rank(method='dense').astype(int)
combined_df = combined_df[['meeting_id'] + [col for col in combined_df.columns if col != 'meeting_id']]
combined_df.to_csv('raw_data/combined_dates.csv', index = False)

In [53]:
combined_df

,meeting_id,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,...,T+6,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15
0,1,2000-02-02,^GSPC,statement,<NA>,0.000522,-0.007095,-0.002912,<NA>,<NA>,...,0.012273,-0.020815,0.003627,-0.020969,<NA>,<NA>,0.002033,0.008713,-0.010256,0.000425
1,1,2000-02-02,^TNX,statement,<NA>,-0.003408,0.00342,0.00489,<NA>,<NA>,...,-0.004826,0.010608,-0.001949,-0.005258,<NA>,<NA>,-0.012385,0.001988,-0.000763,0.00336
2,1,2000-02-02,^IRX,statement,<NA>,0.022945,-0.009346,0.0,<NA>,<NA>,...,0.009141,-0.005435,0.001822,-0.003636,<NA>,<NA>,-0.00365,0.021978,-0.005376,0.003604
3,1,2000-02-02,XLC,statement,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1,2000-02-02,XLK,statement,<NA>,0.001765,0.005868,0.001167,<NA>,<NA>,...,0.009132,-0.012443,0.023769,-0.026573,<NA>,<NA>,0.000574,0.002872,-0.005154,0.022452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7393,408,2025-04-09,^DJI,intermeeting,0.000098,-0.003116,-0.003653,-0.016922,<NA>,<NA>,...,-0.003845,-0.017329,-0.013289,<NA>,<NA>,<NA>,-0.024828,0.026632,0.010707,<NA>
7394,408,2025-04-09,^IXIC,intermeeting,0.004578,-0.020405,-0.005307,-0.027019,<NA>,<NA>,...,-0.000494,-0.030673,-0.00127,<NA>,<NA>,<NA>,-0.025515,0.027063,0.025007,<NA>
7395,408,2025-04-09,^IRX,intermeeting,0.0,0.001913,-0.000477,0.0,<NA>,<NA>,...,0.001192,0.001905,-0.000713,<NA>,<NA>,<NA>,0.001189,0.000713,-0.001187,<NA>
7396,408,2025-04-09,XLU,intermeeting,-0.015991,0.00663,-0.000258,0.007363,<NA>,<NA>,...,-0.000129,-0.009011,0.010262,<NA>,<NA>,<NA>,-0.023659,0.027262,0.004231,<NA>


# Sentiment Analysis 

I just copied the inputs file from the midterm so we can use the ML and LM dictionaries through that......

Shouldn't be too difficult for that part

Then we need to do the topic analysis as well......

## The tricky parts - Be working on this by tuesday or we gonna be in trouble 

### Chat GPT API integration to rank documents on bullish to bearish scale

### ChronoBERT - yikes 